Let's start again with the classical imports and shortcuts.

In [3]:
from robots import loadTalosArm
from scipy.optimize import fmin_slsqp
import pinocchio
from pinocchio.utils import *
from numpy.linalg import norm,inv,pinv,eig,svd

m2a = lambda m: np.array(m.flat)
a2m = lambda a: np.matrix(a).T

robot   = loadTalosArm()
robot.initDisplay(loadModel=True)
robot.viewer.gui.deleteNode('world',True)
robot.initDisplay(loadModel=True)

rmodel = robot.model
rdata = rmodel.createData()

# Dynamics with 3 main algorithms
First recall that the dynamics of a robot can be written under the Langrangian form by:

$M(q) \dot{v}_q + c(q,v_q) + g(q) = $tau_q$

where $q, v_q, \dot{v}_q$ are the position, velocity and acceleration in the configuration space (that we can denote by $q,\dot{q},\ddot{q}$ in easy cases), $c$ are the centrifugal and Coriolis terms, $g$ is the gravity force in the configuration space and $\tau_q$ are the torques applied at the motor by any other means, in particular by the motors, by possible springs and by the external forces. The $c$ is quadratic and is sometime written $c(q,v_q) = C(q,v_q) v_q = v_q^T R(q) v_q$ with $C$ a matrix and $R$ a tensor.

As $M$ is a positive definite matrix (otherwise it means that you have a problem with your robot model), we can invert it and also write it in the other way, i.e as $\dot{v}_q$ a function $q,v_q,\tau_q$.

$M^-1(q) ( \tau_q - c(q,v_q) - g(q) ) = \dot{v}_q$

We call the first equation the inverse dynamics (it is typically a control viewpoint) and the second the direct/forward dynamics (it is rather a simulation viewpoint). 

In [4]:
q = pinocchio.randomConfiguration(rmodel)
vq = rand(rmodel.nv)*2-1
aq = rand(rmodel.nv)*2-1

## The inverse dynamics: Recursive Newton Euler Algorithm

The RNE algorithm compute the inverse dynamics ie $\tau_q :=rnea(q,v_q,\dot{v}_q) $

In [7]:
tauq = pinocchio.rnea(rmodel,rdata,q,vq,aq)
print(tauq.T)

[[ 1.67600113e+01  7.57439465e+00 -7.64209909e+00  5.51064376e-01
   1.20253731e-01 -1.79974541e+00  6.23997782e-03]]


## The forward dynamics: Articulated Body Algorithm
The AB algorithm compute the forward dynamics ie $\dot{v}_q = aba(q,v_q,\tau_q)$

In [9]:
aq_check = pinocchio.aba(rmodel,rdata,q,vq,tauq)
print(norm(aq-aq_check))

8.564937111781086e-14


## Computing the mass matrix: the Composite Rigid Body Algorithm
The CRB algorithm compute $M$ as a function of $q$.

In [11]:
M = pinocchio.crba(rmodel,rdata,q)
print(eig(M)[0])

[0.99825852 0.33533915 0.08279344 0.01130456 0.00827147 0.00447846
 0.00110745]


When you need M, you also typically need $b(q,v_q) = c(q,v_q)+g(q)$ that we sometime name the biais, or drift, or affine term of the dynamics. You can compute both with:

In [13]:
pinocchio.computeAllTerms(rmodel,rdata,q,vq)
print(norm(M-rdata.M))
nle_check = pinocchio.rnea(rmodel,rdata,q,vq,0*aq)
print(norm(rdata.nle-nle_check))

0.0
1.7798789105919607e-15


ComputeAllTerms does not returns anything but computes and stores many interesting values in rdata. It also compute the rdata.com and its derivative rdata.Jcom, the kinetic and potential energy, etc.

These 4 algorithms also compute all the joint placements, velocities, accelerations and forces, hence corresponding to more than a call to forwardKinematics(q,vq,aq).


## Forces in RNEA and ABA

We may often want to separate $\tau_q$ in two terms $\tau_q = \tau_m+J^T \phi$, where $\tau_m$ are the forces due to the robot itself (motors, elastics, etc) and $J^T \phi$ are the joint torques due to the external forces $\phi$, where $J$ is the Jacobian of the contact interfaces. The most complete case is when $\phi$ is a spatial force; then $J$ is the 6D Jacobian that transforms the configuration velocity into the velocity of the contact frame (both velocity and force being represented in the same space, typically LOCAL).

Both RNEA and ABA accept forces in their interface as a STD vector of spatial forces. The interface here is not super nice, as you should give a comprehensive list of all possible forces, putting 0 when you don't want a force.


In [16]:
forces = pinocchio.StdVect_Force()
for i in range(rmodel.njoints): forces.append(pinocchio.Force.Zero())
pinocchio.rnea(rmodel,rdata,q,vq,aq,forces)
pinocchio.aba(rmodel,rdata,q,vq,tauq,forces);

# Contact inverse dynamics optimization problem
This problem is to find the torque and forces that makes a given acceleration (for example $\dot{v}_q =0$) possible.

$min_{phi,\tau_q} || M \dot{v}_q + b - \tau_q - J^T \phi ||^2$

Typically, you want to constraint $\phi$ and $\tau_q$ to be in some given domain (like $\phi$ in the contact cones and $\tau_q$ in the motor range). As pinocchio is not implementing a contact model that would define such ranges, we do not treat the problem here. And typically, you hope to nullify the cost.

Writing the problem when contact forces are computed at some joint frames is relatively easy. Let's do it directly when expressing quantities at some other operational frames, which is slightly more tricky and also more useful.

First, recall that you can get a frame ID and the corresponding Jacobian as follows:


In [19]:
fid = rmodel.getFrameId('gripper_left_fingertip_2_link')
LOCAL = pinocchio.ReferenceFrame.LOCAL
pinocchio.computeAllTerms(rmodel,rdata,q,vq)
pinocchio.updateFramePlacements(rmodel,rdata)
pinocchio.getFrameJacobian(rmodel,rdata,fid,LOCAL)

matrix([[ 3.58630743e-01,  2.92269729e-01, -3.85095748e-01,
          3.13637554e-02, -9.56834956e-04, -2.13985000e-01,
          0.00000000e+00],
        [ 2.98451208e-01,  3.50288617e-01,  2.60708248e-01,
         -3.34621874e-02,  2.13348819e-01, -3.46944695e-18,
          0.00000000e+00],
        [ 3.03169295e-02,  9.71650125e-02,  1.28190255e-01,
         -1.26873212e-02,  5.11310480e-02,  3.20000000e-02,
          0.00000000e+00],
        [ 1.15802973e-01,  7.47386037e-01,  6.01407197e-01,
         -1.73920705e-02,  9.99824951e-01, -2.16840434e-17,
          0.00000000e+00],
        [-3.83311369e-02, -6.54797986e-01,  7.38722116e-01,
         -3.68684086e-01, -5.03069808e-17,  1.00000000e+00,
          0.00000000e+00],
        [-9.92532314e-01,  1.12488707e-01,  3.04300869e-01,
          9.29392038e-01,  1.87101086e-02,  1.38777878e-17,
          0.00000000e+00]])

We now have M, b, and the J, so writing the optimization problem is straight forward. Just do it!
Below is a solution with the biped and 2 contact points if you need.

In [29]:
# %load solution_cid.py

A sanity check can be implemented using RNEA. But for that, we cannot keep the forces expressed in the contact frame: RNEA only has an interface to accept forces expressed in joint frames. This is not difficult to change: the frames objects in rmodel contain their placement with respect to the joint parent: 

In [30]:
rmodel.frames[fid].placement

  R =
1 0 0
0 1 0
0 0 1
  p = 0 0 0

The force can be changed by applying this transform to the spatial force. Indeed SE3 placements implement an "act" interface that act on other spatial objects, in particular on forces.

In [31]:
M = rmodel.frames[fid].placement
f = pinocchio.Force.Random()
M.act(f)

  f = -0.514226 -0.725537  0.608354
tau = -0.686642 -0.198111 -0.740419

For all your contact points, get the transform from contact frame to joint parent frame, and transform the forces resulting from the optimization scheme into forces expressed in the joint parent. Then call RNEA to check that the pair $\tau_q,\phi$ is solving the dynamics equation.

In [32]:
# %load solution_cid_check.py
